In [4]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import iaaft
import xarray as xr
import pandas as pd

In [2]:
def create_new_dataset(original_ds, start_date='1970-01-01'):
    num_days = original_ds.dims['time']  # Assumendo che 'time' sia la dimensione temporale
    dates = pd.date_range(start=start_date, periods=num_days, freq='D')

    new_ds = xr.Dataset(
        {
            't2m': (('time', 'lat', 'lon'), np.full((num_days, len(original_ds['lat']), len(original_ds['lon'])), np.nan))
        },
        coords={
            'lat': original_ds['lat'],
            'lon': original_ds['lon'],
            'time': ('time', dates)
        }
    )
    
    new_ds['time'].attrs['units'] = 'days since 1970-01-01 00:00:00'
    #new_ds['t2m'].attrs = original_ds['t2m'].attrs  # Copia gli attributi se necessario

    return new_ds

def compute_anomalies(ds, VARIABLE, BASELINE_INTERVAL):
    start, end = BASELINE_INTERVAL
    ds = ds[VARIABLE]
    ds_baseline = ds.sel(time=(ds['time.year'] >= start) & (ds['time.year'] <= end))
    
    gb = ds_baseline.groupby('time.dayofyear')
    clim = gb.mean(dim='time')
    std_clim = gb.std(dim='time')
    
    clim_time = clim.sel(dayofyear=ds.time.dt.dayofyear)
    std_clim_time = std_clim.sel(dayofyear=ds.time.dt.dayofyear)

    anomalies = (ds - clim_time)
    return anomalies

In [5]:
# Carica il dataset originale
original_ds = xr.open_dataset("D:/HD climatenetwork/PAPER/t2m_1970_2022_5grid.nc")
new_ds = create_new_dataset(original_ds)
new_ds['t2m'].loc[:,:,:] = original_ds['t2m']

# anomalie baseline 1970, 1989
BASELINE_INTERVAL = [1970, 1989]
anomalies_bs7089 = compute_anomalies(new_ds, 't2m', BASELINE_INTERVAL)


In [15]:
# Crea il nuovo dataset NetCDF4
surr_dataset = nc.Dataset('X:/surr_IAAFT_t2m_1970_2022.nc', 'w', format='NETCDF4')

# Definisci le dimensioni
num_dim = surr_dataset.createDimension('num', 51)   # choose the number of surrogates plus one (index 0 is the original) 
time_dim = surr_dataset.createDimension('time', len(new_ds['time']))  # Assumendo che 'time' sia la dimensione temporale
lat_nuova = surr_dataset.createDimension('lat', len(new_ds['lat']))
lon_nuova = surr_dataset.createDimension('lon', len(new_ds['lon']))

# Crea le variabili
time_var = surr_dataset.createVariable('time', np.float32, ('time',))
lat_var = surr_dataset.createVariable('lat', np.float32, ('lat',))
lon_var = surr_dataset.createVariable('lon', np.float32, ('lon',))
num_var = surr_dataset.createVariable('num', np.int32, ('num',))
t2m_var = surr_dataset.createVariable('t2m', np.float32, ('num', 'time', 'lat', 'lon'))
lat_var[:] = new_ds['lat'][:]
lon_var[:] = new_ds['lon'][:]


num_days = len(new_ds['time'])
start_date = '1970-01-01'
dates = pd.date_range(start=start_date, periods=num_days, freq='D')
time_var.units = 'days since 1970-01-01 00:00:00'
time_var.calendar = 'standard'
time_var[:] = nc.date2num(dates.to_pydatetime(), units=time_var.units, calendar=time_var.calendar)


print(lat_var)
print(lon_var)
print(surr_dataset.variables.keys())
print(num_var)
print(t2m_var)


<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (37,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (72,)
filling on, default _FillValue of 9.969209968386869e+36 used
dict_keys(['time', 'lat', 'lon', 'num', 't2m'])
<class 'netCDF4._netCDF4.Variable'>
int32 num(num)
unlimited dimensions: 
current shape = (51,)
filling on, default _FillValue of -2147483647 used
<class 'netCDF4._netCDF4.Variable'>
float32 t2m(num, time, lat, lon)
unlimited dimensions: 
current shape = (51, 19358, 37, 72)
filling on, default _FillValue of 9.969209968386869e+36 used


In [29]:
lat_var = surr_dataset.variables['lat']
lon_var = surr_dataset.variables['lon']
num_var = surr_dataset.variables['num']
sh_t2m = surr_dataset.variables['t2m']
lon_range = range(0, len(lon_var))
lat_range = range(0, len(lat_var))


# PARAMETERS
num_surr = 50
start_year = 1970
end_year = 2022
last_day_index = 19358

#start_year = 2022
#end_year = 2100
#last_day_index = 28854

tot_years = end_year - start_year + 1


# Inizializza la lista dei giorni di inizio
start_days = []

current_day = 0 
for year in range(start_year, end_year + 1):
    start_days.append(current_day)
    # Controlla se l'anno è bisestile
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        days_in_year = 366
    else:
        days_in_year = 365
    current_day += days_in_year


start_days.append(last_day_index)

### IAAFT Surrogates
                

for i in lat_range:
    for j in lon_range:
        print(f'Surrogating node ({i},{j}) :')
        #original data in index 0
        selected_data = anomalies_bs7089.isel(lat=i, lon=j)
        sh_t2m[0,:,i,j] = selected_data

        # surrogates generation
        all_surrogates = [[] for _ in range(num_surr)]
        for dec in range(tot_years):
            
            sh_serieiaaft1 = iaaft.surrogates(x=selected_data[start_days[dec]:start_days[dec+1]], ns=num_surr, verbose=False )

            for n in range(num_surr):
                for value in sh_serieiaaft1[n]:
                    all_surrogates[n].append(value)  # Assuming this is the surrogate data generated

        for n2 in range(num_surr):                                        
            sh_t2m[n2+1,:, i, j] = all_surrogates[n2]


# Chiudi il dataset
surr_dataset.close()

Surrogating node (0,0) :
Surrogating node (0,1) :
Surrogating node (0,2) :
Surrogating node (0,3) :
Surrogating node (0,4) :
Surrogating node (0,5) :
Surrogating node (0,6) :
Surrogating node (0,7) :
Surrogating node (0,8) :
Surrogating node (0,9) :
Surrogating node (0,10) :
Surrogating node (0,11) :
Surrogating node (0,12) :
Surrogating node (0,13) :
Surrogating node (0,14) :
Surrogating node (0,15) :
Surrogating node (0,16) :
Surrogating node (0,17) :


KeyboardInterrupt: 